## Importar librerías necesarias

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

## Carga de datos

In [2]:
df = pd.read_csv("data/spam.csv",encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Dummies spam y mensaje

In [3]:
df["spam"] = np.where(df["v1"]=='spam',1,0)

In [4]:
df_clean= df[["v2","spam"]]
df_clean.columns = ["message","spam"]
df_clean.head()

,message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


## Cantidad de Datos

In [5]:
df_clean.shape[0]

5572

## Divido el Dataset para saber la proporción entre mensajes con span y normales

In [6]:
spam_df = df_clean.loc[df_clean["spam"]==1]
ham_df = df_clean.loc[df_clean["spam"]==0]
print(f"Cantidad de spam y ham (no spam):\nCantidad de Spam: {spam_df.shape[0]}\nCantidad de no Spam: {ham_df.shape[0]}")

Cantidad de spam y ham (no spam):
Cantidad de Spam: 747
Cantidad de no Spam: 4825


In [7]:
vectorizer_spam = TfidfVectorizer(stop_words='english',max_features=30)
vectorizer_spam.fit(spam_df["message"])
vectorizer_spam.vocabulary_

{'free': 8,
 'win': 27,
 'text': 20,
 'txt': 22,
 'week': 26,
 'send': 17,
 '50': 3,
 'prize': 15,
 'claim': 5,
 'mobile': 11,
 'cash': 4,
 '150p': 0,
 '16': 1,
 'reply': 16,
 'urgent': 25,
 'won': 28,
 'www': 29,
 'com': 6,
 'ur': 24,
 'uk': 23,
 'msg': 12,
 'nokia': 14,
 'new': 13,
 '18': 2,
 'service': 18,
 'guaranteed': 9,
 'contact': 7,
 'stop': 19,
 'just': 10,
 'tone': 21}

## Escogemos la palabra Cash para usar en la frmula

In [8]:
from functools import partial, reduce
import operator as op
word = "cash"
def word_in_text(x,word):
    if word in x[1].values[0]:
        return 1
    else:
        return 0

calculamos $p(w|s)$

In [9]:
spam_word_count = reduce(op.add,list(map(partial(word_in_text,word=word),spam_df.iterrows())))
spam_count = spam_df.shape[0]

In [10]:
prob_of_word_given_spam = op.truediv(spam_word_count,spam_count)
print(f"probabilidad p(w|s) : {round(prob_of_word_given_spam,6)*100}%")

probabilidad p(w|s) : 6.8273%


calculamos $p(w)$

In [11]:
msg_word_count = reduce(op.add,list(map(partial(word_in_text,word=word),df_clean.iterrows())))
msg_count = df_clean.shape[0]

In [12]:
prob_of_word = op.truediv(msg_word_count,msg_count)
print(f"probabilidad p(w) : {round(prob_of_word,6)*100}%")

probabilidad p(w) : 1.1486%


calculamos $p(s)$

In [13]:
prob_of_spam = op.truediv(spam_count,df_clean.shape[0])
print(f"probabilidad p(s) : {round(prob_of_spam,6)*100}%")

probabilidad p(s) : 13.406299999999998%


Se quiere calcular $p(s|w) $

Por el teorema de Bayes se sabe que esto es igual a:

$$ p(spam|cash) = \frac{p(cash|spam)* p(spam) }{p(cash)} $$

In [14]:
prob_s_w  = op.truediv(op.mul(prob_of_word_given_spam,prob_of_spam),prob_of_word) 
print(f"La probabilidad de que el mensaje sea spam es:\n{round(prob_s_w,6)*100}%")

La probabilidad de que el mensaje sea spam es:
79.6875%


Podemos probar múltiples combinaciones de palabras y obtener las probabilidades individuales de qué nos digan que tan probable son aquellas combinaciones, como lo han visto ahora. Refinando esto y haciéndonos la siguiente pregunta: Dada una cantidad de palabras, ¿Cuál es la probabilidad de que el mensaje sea spam dado ese conjunto de palabras? 

**Lo podemos calcular con la siguiente fórmula:**

carlos.schoenfeldt@gmail.com